In [ ]:
from timepix_geometry_correction.correct import TimepixGeometryCorrection
import os

from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
from tqdm import tqdm

import glob

In [ ]:
sample = "/SNS/VENUS/IPTS-35423/shared/autoreduce/images/tpx1/raw/radiography/20251113_1_GdFe__5_000C_0_700AngsMin/20251113_Run_14493_1_GdFe__5_000C_0_700AngsMin_0"
assert os.path.exists(sample)

ob = "/SNS/VENUS/IPTS-35423/shared/autoreduce/images/tpx1/ob/20251113_OB__5_000C_0_700AngsMin/20251113_Run_14490_OB__5_000C_0_700AngsMin_ob_0"
assert os.path.exists(ob)

In [ ]:
# load the sample and ob and integrate them
list_sample_tiff = glob.glob(os.path.join(sample, "*.tif"))
assert len(list_sample_tiff) > 0, f"No tiff files found in {sample}"
list_ob_tiff = glob.glob(os.path.join(ob, "*.tif"))
assert len(list_ob_tiff) > 0, f"No tiff files found in {ob}"

In [ ]:
sample_data = []
for tiff in tqdm(list_sample_tiff):
    img = Image.open(tiff)
    sample_data.append(np.array(img))
sample_data = np.array(sample_data)
integrated_sample = np.sum(sample_data, axis=0)

ob_data = []
for tiff in tqdm(list_ob_tiff):
    img = Image.open(tiff)
    ob_data.append(np.array(img))
ob_data = np.array(ob_data)
integrated_ob = np.sum(ob_data, axis=0)



In [ ]:
# preview
fig, axs = plt.subplots(1, 2, figsize=(10, 15))
axs[0].imshow(integrated_sample, cmap="viridis")
axs[0].set_title("Integrated Sample")
axs[1].imshow(integrated_ob, cmap="viridis")
axs[1].set_title("Integrated OB")
plt.show()

In [ ]:
normalized = integrated_sample / integrated_ob
plt.imshow(normalized, cmap="viridis")
plt.title("Normalized Sample")
plt.colorbar()
plt.show()

In [ ]:
o_corrector_sample = TimepixGeometryCorrection(raw_images=[integrated_sample])
corrected_sample = o_corrector_sample.correct(display=False)

o_corrector_ob = TimepixGeometryCorrection(raw_images=[integrated_ob])
corrected_ob = o_corrector_ob.correct(display=False)


In [ ]:
corrected_sample = corrected_sample[0]
corrected_ob = corrected_ob[0]

In [ ]:
# display results
fig, axs = plt.subplots(1, 2, figsize=(15, 15))
axs[0].imshow(corrected_sample, cmap="viridis")
axs[0].set_title("corrected Sample")
axs[1].imshow(corrected_ob, cmap="viridis") 
axs[1].set_title("corrected OB")

In [ ]:
# normalization of corrected data
normalized_corrected = corrected_sample / corrected_ob
plt.figure(figsize=(15, 15))
plt.imshow(normalized_corrected, cmap="viridis", vmin=0, vmax=1)
plt.title("Normalized Corrected Sample")
plt.colorbar()
plt.show()

In [ ]:
# to check that if we do not correct the OB, we get the bad result

In [ ]:
# # normalization of corrected data
# normalized_corrected = corrected_sample / integrated_ob
# plt.figure(figsize=(10, 15))
# plt.imshow(normalized_corrected, cmap="viridis", vmin=0, vmax=1)
# plt.title("Normalized Corrected Sample")
# plt.colorbar(shrink=0.5)
# plt.show()